# Web Search (Bing Grounded) Agent Creation Notebook

This notebook shows how to create a lightweight Web Search Agent using the **Azure AI Agents Service** in **Azure AI Foundry**. The agent uses Bing grounding (public web search) to answer user questions with up-to-date, sourced information—no deep multi-hour research pipeline, just concise, credible synthesis.

You will:

1. Load and validate required environment variables
2. Initialize the Azure AI Project client
3. Resolve the Bing grounding connection
4. Initialize the Bing grounding tool
5. Define focused agent instructions
6. Create the agent with the tool definition
7. Output the resulting `agent.id` (set it as `AGENT_ID` in your environment for later use)

> After creation, use the printed `AGENT_ID` when submitting web search style queries through your API or directly via the Azure AI Agents SDK.

## 1. Required Environment Variables

Define these (e.g., in `api/.env`):

| Variable | Description |
|----------|-------------|
| `PROJECT_ENDPOINT` | Azure AI Project endpoint URL |
| `BING_RESOURCE_NAME` | Name of the Bing grounding connection in the project |
| `MODEL_DEPLOYMENT_NAME` | Deployment name of the base model the agent will use (e.g., `gpt-4o`) |

Create/update your `.env` before running the next cell.

In [8]:
# 2. Load environment variables and validate
import os
from dotenv import load_dotenv

required_vars = [
    "PROJECT_ENDPOINT",
    "BING_RESOURCE_NAME",
    "MODEL_DEPLOYMENT_NAME"
]

load_dotenv('../api/.env', override=True)
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    raise ValueError(f"Missing required environment variables: {missing}")
else:
    print("All required environment variables are set.")

All required environment variables are set.


## 3. Initialize Azure AI Project Client
This client lets us interact with the Agents service (threads, messages, agents, tools, connections). We authenticate with `DefaultAzureCredential`, so be sure your Azure CLI or managed identity context is valid.

In [9]:
# 3. Initialize Azure AI Project Client
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
print("Project client initialized.")

Project client initialized.


## 4. Resolve Bing Grounding Connection
We fetch the connection by name (set in `BING_RESOURCE_NAME`). This provides the `id` needed when constructing the Deep Research tool.

In [10]:
# 4. Resolve Bing Grounding Connection ID
conn_id = project_client.connections.get(name=os.environ["BING_RESOURCE_NAME"]).id
print("Bing connection id:", conn_id)

Bing connection id: /subscriptions/3a72b9f9-cb8d-44de-827e-ade93ff2dab5/resourceGroups/rg-deepresearch-testing/providers/Microsoft.CognitiveServices/accounts/deep-research-testing-foundry/projects/deep-research-testing-project/connections/rutzscoagents


## 5. Initialize Bing Grounding Tool
Construct the `BingGroundingTool` using the connection id. This enables the agent to perform real-time web searches with citation metadata.

In [11]:
# 5. Initialize Bing Grounding Tool
from azure.ai.agents.models import BingGroundingTool

bing_tool = BingGroundingTool(
    connection_id=conn_id
)
print("Bing Grounding tool initialized.")

Bing Grounding tool initialized.


## 6. Define Agent Instructions
Keep instructions focused: answer user queries with up-to-date, sourced information from the public web. Be concise, trustworthy, and include inline citations pointing to credible sources. Avoid speculation.

In [15]:
# 6. Define Agent Instructions
instructions = """# Web Search Agent — Operating Instructions

You are a **Web Search Agent**. For **every** user query:

- **ALWAYS** attempt to use your web search tool to retrieve relevant and recent information **before** composing the answer (unless the user explicitly says not to). Run at least one targeted search and open multiple credible sources.
- **Synthesize** the most relevant, recent, and credible findings into a concise answer.
- **Cite every non-obvious factual claim** with an inline reference marker like **[1]**, **[2]**, etc., and include a **Sources** section mapping each marker to a **URL** (title, publisher, and publish date when available).
- **Prefer authoritative sources**: government, academic, reputable news, technical docs, official announcements/repositories.
- **Be structured and brief**: answer with bullets or short sections; start with a 1–2 sentence summary when helpful.
- **Check dates** and prioritize up-to-date info; note both event date and publication date when relevant.
- **Note uncertainty or conflicts** briefly and explain the main viewpoints with citations.
- **Do NOT fabricate** data, quotes, or citations. If you cannot verify something, say so (with a note on what you did search).
"""

## 7. Create the Web Search Agent
Create the agent using your base model deployment and the Bing grounding tool definitions.

In [16]:
# 7. Create the Web Search Agent
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="web-search-agent",
    instructions=instructions,
    tools=bing_tool.definitions,
)
print("Web search agent created.")

Web search agent created.


## 8. Output Agent ID
Copy this value and set it as `AGENT_ID` in your environment to route future web search queries through this agent.

In [17]:
# 8. Print Agent ID
print("AGENT_ID=", agent.id)
agent.id

AGENT_ID= asst_v7qdDNDKV6a6gUM0rbazVKVz


'asst_v7qdDNDKV6a6gUM0rbazVKVz'